In [1]:
!curl -Ls https://github.com/iden3/circom/releases/latest/download/circom-linux-amd64 -o circom
!chmod +x circom
!sudo mv circom /bin/circom

!npm install -g snarkjs
!npm install circomlib

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴
added 45 packages in 8s
⠴⠙⠹⠸⠼⠴⠦⠧
added 1 package in 929ms
⠧

In [2]:
%%writefile temperature.circom
pragma circom 2.0.0;

include "node_modules/circomlib/circuits/poseidon.circom";
include "node_modules/circomlib/circuits/comparators.circom";

template TemperatureCheck(n) {
    // --- Inputs ---
    signal input readings[n];     // The private temperature log
    signal input secretSalt;      // Private salt to secure the hash
    signal input maxTemp;         // Public rule (e.g., 8 degrees)

    // --- Outputs ---
    signal output dataHash;       // Public anchor (links to Smart Contract)
    signal output isBreach;       // Public result (1 = breach, 0 = safe)

    // --- 1. Integrity Check (Calculate Hash) ---
    // We use Poseidon hash (standard in ZK) to fingerprint the data
    component hasher = Poseidon(n + 1);
    for (var i = 0; i < n; i++) {
        hasher.inputs[i] <== readings[i];
    }
    hasher.inputs[n] <== secretSalt;
    dataHash <== hasher.out;

    // --- 2. Compliance Check (Logic) ---
    signal breachCount[n+1];
    breachCount[0] <== 0;

    component gt[n];

    for (var i = 0; i < n; i++) {
        // Check if readings[i] > maxTemp
        gt[i] = GreaterThan(64);
        gt[i].in[0] <== readings[i];
        gt[i].in[1] <== maxTemp;

        // Accumulate breaches
        breachCount[i+1] <== breachCount[i] + gt[i].out;
    }

    // If breachCount > 0, then isBreach = 1, else 0
    component isPositive = GreaterThan(64);
    isPositive.in[0] <== breachCount[n];
    isPositive.in[1] <== 0;

    isBreach <== isPositive.out;
}

// We define a circuit with 5 readings for this demo
component main {public [maxTemp]} = TemperatureCheck(5);

Writing temperature.circom


In [3]:
# 1. Compile the circuit
!circom temperature.circom --r1cs --wasm --sym --c

# 2. Generate Trusted Setup (Phase 1) LOCALLY
# We generate a small setup (Power 12) which supports up to 4096 constraints.
!snarkjs powersoftau new bn128 12 pot12_0000.ptau -v
!snarkjs powersoftau contribute pot12_0000.ptau pot12_final.ptau --name="FirstContribution" -v -e="random text"
!snarkjs powersoftau prepare phase2 pot12_final.ptau pot12_final_prepared.ptau -v

# 3. Circuit Specific Setup (Phase 2)
!snarkjs groth16 setup temperature.r1cs pot12_final_prepared.ptau circuit_0000.zkey
!snarkjs zkey contribute circuit_0000.zkey circuit_final.zkey --name="SecondContribution" -v -e="more random text"
!snarkjs zkey export verificationkey circuit_final.zkey verification_key.json

# 4. Export the Verifier Contract
!snarkjs zkey export solidityverifier circuit_final.zkey Verifier.sol

# 5. Verify it worked
!ls -l Verifier.sol

template instances: 81
non-linear constraints: 747
linear constraints: 633
public inputs: 1
private inputs: 6
public outputs: 2
wires: 1382
labels: 2039
Written successfully: ./temperature.r1cs
Written successfully: ./temperature.sym
Written successfully: ./temperature_cpp/temperature.cpp and ./temperature_cpp/temperature.dat
Written successfully: ./temperature_cpp/main.cpp, circom.hpp, calcwit.hpp, calcwit.cpp, fr.hpp, fr.cpp, fr.asm and Makefile
Written successfully: ./temperature_js/temperature.wasm
Everything went okay
[DEBUG] snarkJS: Calculating First Challenge Hash
[DEBUG] snarkJS: Calculate Initial Hash: tauG1
[DEBUG] snarkJS: Calculate Initial Hash: tauG2
[DEBUG] snarkJS: Calculate Initial Hash: alphaTauG1
[DEBUG] snarkJS: Calculate Initial Hash: betaTauG1
[DEBUG] snarkJS: Blank Contribution Hash:
		786a02f7 42015903 c6c6fd85 2552d272
		912f4740 e1584761 8a86e217 f71f5419
		d25e1031 afee5853 13896444 934eb04b
		903a685b 1448b755 d56f701a fe9be2ce
[INFO]  snarkJS: First Contrib

In [4]:
%%writefile generate_proof.js
const snarkjs = require("snarkjs");
const fs = require("fs");

async function run() {
    console.log("--- Generating Proof for 'Proof-of-Cold' ---");

    // 1. SIMULATE DATA (The "Diary")
    const readings = [7, 6, 8, 10, 7];
    const maxTemp = 8;
    const secretSalt = "12345";

    const input = {
        "readings": readings,
        "secretSalt": secretSalt,
        "maxTemp": maxTemp
    };

    console.log("Data:", input);

    // 2. CALCULATE PROOF
    const { proof, publicSignals } = await snarkjs.groth16.fullProve(
        input,
        "temperature_js/temperature.wasm",
        "circuit_final.zkey"
    );

    // 3. PREPARE FOR REMIX
    function p2sol(argv) {
        return {
            a: [argv[0], argv[1]],
            b: [[argv[0][0], argv[0][1]], [argv[1][0], argv[1][1]]],
            c: [argv[0], argv[1]]
        };
    }

    const solidityProof = p2sol([
        proof.pi_a.slice(0, 2),
        proof.pi_b.slice(0, 2),
        proof.pi_c.slice(0, 2),
    ]);

    console.log("\n\n=== COPY THESE INPUTS INTO REMIX 'settleClaim' ===");
    console.log("\nproductId: 1");
    console.log("\na (uint[2]):");
    console.log(JSON.stringify(solidityProof.a));
    console.log("\nb (uint[2][2]):");
    console.log(JSON.stringify(solidityProof.b));
    console.log("\nc (uint[2]):");
    console.log(JSON.stringify(solidityProof.c));
    console.log("\ninput (uint[2]):");
    console.log(JSON.stringify(publicSignals));

    console.log("\n-----------------------------------------------");
    console.log("NOTE: Ensure your 'rootOfRoots' in SupplyChain.stopAndSeal match publicSignals[0]!");
    console.log("Root (Decimal): " + publicSignals[0]);
    console.log("Root (Hex): " + BigInt(publicSignals[0]).toString(16));
}

run().then(() => {
    process.exit(0);
});

Writing generate_proof.js


In [5]:
!npm install snarkjs

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏
added 45 packages, and audited 47 packages in 3s
⠏
found 0 vulnerabilities
⠏

In [6]:
!node generate_proof.js

--- Generating Proof for 'Proof-of-Cold' ---
Data: { readings: [ 7, 6, 8, 10, 7 ], secretSalt: '12345', maxTemp: 8 }


=== COPY THESE INPUTS INTO REMIX 'settleClaim' ===

productId: 1

a (uint[2]):
[["16664632541767753416551659124538766663750947478497884288927593184036478499148","15488281074789644701075206887054087572188607153584266335976558717313316221509"],[["17738587949394895011871357918888264307393375910575626412441673686683934232630","4282482723644289253826635364603596696279664435466181797043977719598743584261"],["21778822718067717047058303218570599926259813011008940821030023099593951565121","10435385568477932324263584753901797539701178598701048465833264171063254693510"]]]

b (uint[2][2]):
[["16664632541767753416551659124538766663750947478497884288927593184036478499148","15488281074789644701075206887054087572188607153584266335976558717313316221509"],[["17738587949394895011871357918888264307393375910575626412441673686683934232630","42824827236442892538266353646035966962796644354661

In [7]:
%%writefile generate_proof_fixed.js
const snarkjs = require("snarkjs");
const fs = require("fs");

async function run() {
    const readings = [7, 6, 8, 10, 7];
    const maxTemp = 8;
    const secretSalt = "12345";

    const input = { "readings": readings, "secretSalt": secretSalt, "maxTemp": maxTemp };

    const { proof, publicSignals } = await snarkjs.groth16.fullProve(
        input, "temperature_js/temperature.wasm", "circuit_final.zkey"
    );

    // FIXED: Simple extraction of a, b, c
    const a = [proof.pi_a[0], proof.pi_a[1]];
    const b = [[proof.pi_b[0][1], proof.pi_b[0][0]], [proof.pi_b[1][1], proof.pi_b[1][0]]];
    const c = [proof.pi_c[0], proof.pi_c[1]];

    console.log("\n=== COPY INTO REMIX settleClaim ===");
    console.log("productId: 1");
    console.log("a:"); console.log(JSON.stringify(a));
    console.log("b:"); console.log(JSON.stringify(b));
    console.log("c:"); console.log(JSON.stringify(c));
    console.log("input:"); console.log(JSON.stringify(publicSignals));
}

run().then(() => process.exit(0));

Writing generate_proof_fixed.js


In [8]:
!node generate_proof_fixed.js


=== COPY INTO REMIX settleClaim ===
productId: 1
a:
["895794964240630995565733852327710057008039897640464805458608886966386554246","12505343554276587259548037132645669018898343498804074089044527300737403019383"]
b:
[["6558063226098936751167165652352259415158248653798924489709213671942334914903","1984082709996835477834604565020467614599992209311432949492150908074433076616"],["11873088134291117814028152226814092933439024167225277998898357652354154564478","8726690997807653459216735793382058251680372380137331030882864253971454196038"]]
c:
["4047564263758625846127792601902253701754127821802269573183554705197975411888","3806868632122412879671745672419417399935864945519711408513775692347455939975"]
input:
["18923456999559959399426891084644560221456840635585613059594696360932605345914","1","8"]


In [9]:
%%writefile generate_frontend_proof.js
const snarkjs = require("snarkjs");
const fs = require("fs");

async function run() {
    // DATA SIMULATION
    // We inject a '10' degree reading which violates the maxTemp of '8'
    const readings = [7, 6, 8, 10, 7];
    const maxTemp = 8;
    const secretSalt = "12345";
    const input = { "readings": readings, "secretSalt": secretSalt, "maxTemp": maxTemp };

    // PROOF GENERATION
    const { proof, publicSignals } = await snarkjs.groth16.fullProve(
        input, "temperature_js/temperature.wasm", "circuit_final.zkey"
    );

    // FORMATTING FOR FRONTEND
    const proofForFrontend = {
        a: [proof.pi_a[0], proof.pi_a[1]],
        b: [[proof.pi_b[0][1], proof.pi_b[0][0]], [proof.pi_b[1][1], proof.pi_b[1][0]]],
        c: [proof.pi_c[0], proof.pi_c[1]],
        input: publicSignals
    };

    console.log("\n👇 COPY THE JSON BELOW FOR YOUR FRONTEND 👇");
    console.log("---------------------------------------------------");
    console.log(JSON.stringify(proofForFrontend, null, 2));
    console.log("---------------------------------------------------");

    console.log("\nrootOfRoots (Use this to seal in Remix):");
    console.log("0x" + BigInt(publicSignals[0]).toString(16));
}

run().then(() => process.exit(0));

Writing generate_frontend_proof.js


In [10]:
!node generate_frontend_proof.js


👇 COPY THE JSON BELOW FOR YOUR FRONTEND 👇
---------------------------------------------------
{
  "a": [
    "17040569333819735766661079545098844339057402971476307841503387035739960919762",
    "3858429826524137381255771581182325154624598222993747481154605516629555147484"
  ],
  "b": [
    [
      "15194709348488049864238349511790969768921357537029068336896443718717898536826",
      "9336122847248655742704998094392359148209514901512550858045170637415638878610"
    ],
    [
      "20468419787186140988805586439582955136526053318257352354959081658680213901437",
      "21128550326149759769024885117063432678531111528369041419449006300315537982001"
    ]
  ],
  "c": [
    "5092556620738211026968125311727543691463272510273070309084180842633604431398",
    "17897224338155449677365386343878134396874863684493718543902551014478990571712"
  ],
  "input": [
    "18923456999559959399426891084644560221456840635585613059594696360932605345914",
    "1",
    "8"
  ]
}
----------------------------------